In [ ]:
# Verify Google gave you GPU runtime and switch using dropdown above if not
!nvidia-smi -L
!nvidia-smi

In [ ]:
# Set up colab paths
temp_colab_path = '/content/invokeai_temp_colab'
persist_colab_path = '/content/drive/MyDrive/invokeai_persist_colab'

# Set if instance should be temporary or persistent
chosen_colab_path = temp_colab_path

# Set up temporary storage if needed
if chosen_colab_path == temp_colab_path:
    import os
    if not os.path.exists(temp_colab_path):
      os.makedirs(temp_colab_path)

# Mount and set up Google drive if needed/isn't already
if chosen_colab_path == persist_colab_path:
  import os
  from google.colab import drive
  if not os.path.exists(persist_colab_path):
    drive.mount('/content/drive')
    if not os.path.exists(persist_colab_path):
      os.makedirs(persist_colab_path)

In [ ]:
# Sign up/accept terms/get tokens at https://huggingface.co/runwayml/stable-diffusion-v1-5 https://huggingface.co/runwayml/stable-diffusion-inpainting https://ngrok.com/
huggingface_token = 'HF_TOKEN_HERE'
ngrok_token = 'NGROK_TOKEN_HERE'

In [ ]:
# Install python 3.9
!sudo apt-get update -y
!sudo apt-get install python3.9 python3-pip python3.9-distutils python3.9-dev libpython3.9-dev python3.9-venv -y

# Change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

# Verify python version
!python --version

# Update pip
%cd {chosen_colab_path}
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall
!python -m pip install --upgrade pip
!pip install --upgrade setuptools

# Handle colab dependencies
!ln -s /usr/local/lib/python3.8/dist-packages/google /usr/local/lib/python3.9/dist-packages/google
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
# Create InvokeAI root
invokeai_root = chosen_colab_path + '/invokeai'
import os
os.environ['INVOKEAI_ROOT'] = invokeai_root
if not os.path.exists(invokeai_root):
  os.makedirs(invokeai_root)

In [ ]:
# Create and prepare virtual environment
%cd {invokeai_root}
!python -m venv .venv --prompt InvokeAI
!source .venv/bin/activate; python -m pip install --upgrade pip
!source .venv/bin/activate; pip install InvokeAI[xformers] --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
# Download and configure models
%cd {invokeai_root}
import os
os.environ['HUGGING_FACE_HUB_TOKEN'] = huggingface_token
!source .venv/bin/activate; invokeai-configure --yes
!sed -i 's/--nsfw_checker/--no-nsfw_checker/' invokeai.init

In [ ]:
# Import custom models
#%cd {invokeai_root}
#!cat /content/drive/MyDrive/stable_diffusion_weights/jenkinsmichpa/jenkinsmichpa.yaml >> configs/models.yaml

In [ ]:
# Install and run NGROK proxy
%cd {chosen_colab_path}
!pip install pyngrok --quiet
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'InvokeAI Public URL: {public_url}')


In [ ]:
# Start site
%cd {invokeai_root}
!source .venv/bin/activate; invokeai --web --host 0.0.0.0